In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split

In [19]:
#Load Dataset
df = pd.read_csv("Augmented_Stockist_data.csv")

# Split the dataset into training (80%) and testing (20%) sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [20]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)


In [21]:
df

,Partner_id,Product_id,Geography,Stockist_Type,Scheme_Type,Sales_Value_Last_Period,Sales_Quantity_Last_Period,MRP,Growth_Percentage,Discount_Applied,Bulk_Purchase_Tendency,New_Stockist,Feedback_Score
0,P1003,Solar Solutions,East,Retailer,Cashback,376425.03,443,47046.08,10.97,7.38,Low,True,4.70
1,P1003,Solar Solutions,West,Distributor,Cashback,61771.12,325,41918.42,17.68,14.96,High,True,3.70
2,P1002,Solar Solutions,West,Wholesaler,Cashback,207420.87,73,39077.33,45.16,18.94,High,False,1.46
3,P1003,Solar Solutions,South,Wholesaler,Cashback,408704.80,185,14999.97,40.79,0.55,High,True,4.19
4,P1002,Solar Solutions,North,Wholesaler,Cashback,63763.23,187,37938.53,37.15,16.52,High,True,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15085,P1003,MCCB,West,Distributor,Seasonal Offer,386646.42,95,46804.58,-0.51,5.95,Low,True,3.44
15086,P1002,MCCB,West,Distributor,Seasonal Offer,170013.80,142,49494.69,37.48,3.06,Medium,False,2.67
15087,P1002,MCCB,East,Retailer,Seasonal Offer,24306.40,405,14076.35,-6.59,7.47,Low,False,2.31
15088,P1001,MCCB,West,Wholesaler,Seasonal Offer,153030.93,245,20960.59,17.67,9.19,High,False,3.44


In [22]:
# # One-hot encoding for Geography and Stockist_Type
# train_df = pd.get_dummies(train_df, columns=["Geography", "Stockist_Type"], dtype=int)

# # Identify geography and stockist type columns
# geo_columns = [col for col in train_df.columns if col.startswith("Geography")]
# stockist_columns = [col for col in train_df.columns if col.startswith("Stockist_Type")]

# if not geo_columns or not stockist_columns:
#     raise ValueError("No Geography or Stockist_Type features found after encoding! Check encoding step.")

# # Ensure Sales_Value_Last_Period does not contain zeros to avoid log(0)
# train_df["Sales_Value_Last_Period"] = train_df["Sales_Value_Last_Period"].replace(0, 1)


In [23]:
# Extract Scheme-Product Relationships from Training Data
# Group products under each scheme from the training set
grouped_schemes = train_df.groupby('Scheme_Type')['Product_id'].apply(set).reset_index()


In [24]:
grouped_schemes

,Scheme_Type,Product_id
0,Cashback,"{Solar Solutions, ACB, AIS, E-House, Controlle..."
1,Loyalty Points,"{Starter, BBT, VCB, Modular Switches, RMU, PSS..."
2,Seasonal Offer,"{Starter, Solar Solutions, Pump Starter and Co..."
3,Volume Discount,"{BBT, Pump Starter and Controller, VCB, ACB, M..."


In [25]:
# Convert Scheme-Product Associations into a Binary Matrix
mlb = MultiLabelBinarizer()
scheme_matrix = pd.DataFrame(
    mlb.fit_transform(grouped_schemes['Product_id']),
    index=grouped_schemes['Scheme_Type'],
    columns=mlb.classes_
)


In [26]:
mlb

MultiLabelBinarizer()

In [27]:
# Compute Jaccard Similarity Matrix
similarity_matrix = pd.DataFrame(index=scheme_matrix.index, columns=scheme_matrix.index, dtype=float)

for i in range(len(scheme_matrix)):
    for j in range(len(scheme_matrix)):
        if i != j:
            similarity_matrix.iloc[i, j] = jaccard_score(scheme_matrix.iloc[i], scheme_matrix.iloc[j])
        else:
            similarity_matrix.iloc[i, j] = 1.0  # Self-similarity


In [28]:
similarity_matrix

Scheme_Type,Cashback,Loyalty Points,Seasonal Offer,Volume Discount
Scheme_Type,,,,
Cashback,1.000000,0.142857,0.416667,0.153846
Loyalty Points,0.142857,1.000000,0.153846,0.300000
Seasonal Offer,0.416667,0.153846,1.000000,0.076923
Volume Discount,0.153846,0.300000,0.076923,1.000000


In [29]:
print(scheme_matrix.head())

                 ACB  AIS  BBT  Controller  E-House  MCCB  Modular Switches  \
Scheme_Type                                                                   
Cashback           1    1    0           1        1     1                 0   
Loyalty Points     0    0    1           0        0     0                 1   
Seasonal Offer     0    1    0           1        0     1                 0   
Volume Discount    1    0    1           0        1     0                 1   

                 PSS  Pump Starter and Controller  RMU  SDF  Solar Solutions  \
Scheme_Type                                                                    
Cashback           1                            0    1    1                1   
Loyalty Points     1                            0    1    0                0   
Seasonal Offer     0                            1    0    1                1   
Volume Discount    0                            1    0    0                0   

                 Starter  VCB  VCU  
Scheme_

In [30]:
# Scheme Recommendation Function
def recommend_similar_schemes(scheme_name, top_n=3):
    """
    Recommends the top N similar schemes based on Jaccard similarity scores.
    Parameters:
    scheme_name (str): The name of the scheme for which recommendations are needed.
    top_n (int): The number of similar schemes to return.
    Returns:
    list: A list of the top N recommended schemes.
    """
    if scheme_name not in similarity_matrix.index:
        return f"Scheme '{scheme_name}' not found in data."

    similarities = similarity_matrix.loc[scheme_name].drop(scheme_name)
    recommended_schemes = similarities.sort_values(ascending=False).head(top_n).index.tolist()
    return recommended_schemes



In [31]:
# Compute Scheme Recommendations on Test Set

recommendation = []

for product in test_df["Product_id"].unique():
    # Find all schemes related to this product in the test set
    product_schemes = test_df[test_df["Product_id"] == product]["Scheme_Type"].unique()

    for scheme in product_schemes:
        if scheme in similarity_matrix.index:
            # Get the top 3 similar schemes for the given scheme
            similar_schemes = similarity_matrix.loc[scheme].drop(scheme).sort_values(ascending=False).head(3)

            recommendation.append({
                "Product_id": product,
                "Similarity_Scores": round(similar_schemes.mean(), 6),  # Average similarity score
                "Scheme_1": similar_schemes.index[0] if len(similar_schemes) > 0 else None,
                "Scheme_2": similar_schemes.index[1] if len(similar_schemes) > 1 else None,
                "Scheme_3": similar_schemes.index[2] if len(similar_schemes) > 2 else None,
            })


In [32]:
# Convert Recommendations to DataFrame
recommendation_df = pd.DataFrame(recommendation)

# Merge with original test dataset to include Partner_id for better context
recommendation_df = test_df[["Partner_id", "Product_id"]].drop_duplicates().merge(
    recommendation_df, on="Product_id", how="inner"
)


In [33]:
recommendation_df

,Partner_id,Product_id,Similarity_Scores,Scheme_1,Scheme_2,Scheme_3
0,P1003,Solar Solutions,0.237790,Seasonal Offer,Volume Discount,Loyalty Points
1,P1003,Solar Solutions,0.215812,Cashback,Loyalty Points,Volume Discount
2,P1002,Solar Solutions,0.237790,Seasonal Offer,Volume Discount,Loyalty Points
3,P1002,Solar Solutions,0.215812,Cashback,Loyalty Points,Volume Discount
4,P1001,Solar Solutions,0.237790,Seasonal Offer,Volume Discount,Loyalty Points
...,...,...,...,...,...,...
85,P1001,PSS,0.198901,Volume Discount,Seasonal Offer,Cashback
86,P1002,PSS,0.237790,Seasonal Offer,Volume Discount,Loyalty Points
87,P1002,PSS,0.198901,Volume Discount,Seasonal Offer,Cashback
88,P1003,PSS,0.237790,Seasonal Offer,Volume Discount,Loyalty Points


In [34]:
# Save the Recommendations
recommendation_df.to_csv("Scheme_Recommendations.csv", index=False)



Evaluation Code

In [1]:
# Import required library
import pandas as pd

In [2]:
# Load the test data (long format — one row per availed scheme per partner)
test_df = pd.read_csv("test_data.csv")

# Load the recommendation data (top 3 recommended schemes per partner)
rec_df = pd.read_csv("Scheme_Recommendations.csv")


In [3]:
# Group by Partner_id to get list of all availed schemes
availed_df = (
    test_df.groupby("Partner_id")["Scheme_Type"]
    .apply(list)
    .reset_index()
    .rename(columns={"Scheme_Type": "Availed_Schemes"})
)

In [4]:
# Combine Scheme_1, Scheme_2, Scheme_3 into a single list column
rec_df["Recommended_Schemes"] = rec_df[["Scheme_1", "Scheme_2", "Scheme_3"]].values.tolist()


In [5]:
# Merge availed and recommended schemes using Partner_id
df_all = pd.merge(
    availed_df,
    rec_df[["Partner_id", "Recommended_Schemes"]],
    on="Partner_id",
    how="left"
)

# Ensure both lists are properly formatted
df_all["Availed_Schemes"] = df_all["Availed_Schemes"].apply(lambda x: x if isinstance(x, list) else [])
df_all["Recommended_Schemes"] = df_all["Recommended_Schemes"].apply(lambda x: x if isinstance(x, list) else [])


In [6]:
# Initialize variables
k_list = [1, 2, 3]
results = []

# Evaluate precision, recall, F1 for each Top-K level
for k in k_list:
    precision_list = []
    recall_list = []

    for _, row in df_all.iterrows():
        actual_set = set(row["Availed_Schemes"])
        recommended_k = row["Recommended_Schemes"][:k]  # Top-K recommendations

        if not actual_set:
            continue  # skip if no availed schemes

        # Count correct predictions in Top-K
        tp = sum([1 for scheme in recommended_k if scheme in actual_set])
        precision = tp / k
        recall = tp / len(actual_set)

        precision_list.append(precision)
        recall_list.append(recall)
        
    # Average the metrics across all partners
    avg_precision = round(sum(precision_list) / len(precision_list), 4) if precision_list else 0
    avg_recall = round(sum(recall_list) / len(recall_list), 4) if recall_list else 0
    f1 = round(2 * avg_precision * avg_recall / (avg_precision + avg_recall), 4) if (avg_precision + avg_recall) else 0

    results.append({
        "Top-K": k,
        "Avg Precision": avg_precision,
        "Avg Recall": avg_recall,
        "Avg F1 Score": f1
    })



In [7]:
# Print Top-K per-scheme evaluation metrics
print("==== Per-Scheme Evaluation (WITH Availed Schemes) ====")
for r in results:
    print(f"\nTop-{r['Top-K']}")
    print(f"  Avg Precision : {r['Avg Precision']}")
    print(f"  Avg Recall    : {r['Avg Recall']}")
    print(f"  Avg F1 Score  : {r['Avg F1 Score']}")


==== Per-Scheme Evaluation (WITH Availed Schemes) ====

Top-1
  Avg Precision : 1.0
  Avg Recall    : 0.25
  Avg F1 Score  : 0.4

Top-2
  Avg Precision : 1.0
  Avg Recall    : 0.5
  Avg F1 Score  : 0.6667

Top-3
  Avg Precision : 1.0
  Avg Recall    : 0.75
  Avg F1 Score  : 0.8571
